## Connect to SAP BTP CloudFoundry and get a user key values

In [1]:
from cloudfoundry_client.client import CloudFoundryClient
import os

### Connect Python CF client using user and password

### Connect Python CF client using the default file, i.e. `~/.cf/config.json`

In [2]:
target_endpoint = 'https://api.cf.eu10-004.hana.ondemand.com'
proxy = dict(http=os.environ.get('HTTP_PROXY', ''), https=os.environ.get('HTTPS_PROXY', ''))
client = CloudFoundryClient(target_endpoint, proxy=proxy, verify=True)
# use the default file, i.e. ~/.cf/config.json
client = CloudFoundryClient.build_from_cf_config()

### Get my SAP AI Core service key

...where the service key name in my SAP BTP subaccount is `vital-aicore-ext-sk1`

In [3]:
my_cf_servicekey=client.v2.service_keys.get_first(name='vital-aicore-ext-sk1')

In [ ]:
display(my_cf_servicekey)

## Connect to my SAP AI Core instance

### Connect to my SAP AI Core instance using the values from the service key

In [5]:
# Load the AICore client
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url      = my_cf_servicekey['entity']['credentials']['serviceurls']['AI_API_URL'] + '/v2', # The present SAP AI Core API version is 2
    auth_url      = my_cf_servicekey['entity']['credentials']['url'] + '/oauth/token', # Suffix to add
    client_id     = my_cf_servicekey['entity']['credentials']['clientid'],
    client_secret = my_cf_servicekey['entity']['credentials']['clientsecret'],
    resource_group='default'
)

### Connect to my SAP AI Core instance using the OS environment variables from the values from the service key

### Check GenAI model deployments in my SAP AI Core resource group

In [6]:
response = ai_core_client.deployment.query()

In [7]:
for deployment in response.resources:
    display(
        {key: deployment.__dict__[key] 
            for key in ('configuration_id', 'configuration_name', 'status', 'details') 
            if deployment.__dict__['scenario_id']=='foundation-models'}
    )


{'configuration_id': '14305345-7a79-45a0-a53a-1cd2f73613bb',
 'configuration_name': 'text-embedding-ada-002-config',
 'status': <Status.RUNNING: 'RUNNING'>,
 'details': {'resources': {'backend_details': {'model': {'name': 'text-embedding-ada-002',
     'version': 'latest'}}},
  'scaling': {'backend_details': {}}}}

{'configuration_id': 'df893fbf-5bae-4481-8a63-a572c83db2e6',
 'configuration_name': 'gpt-4-32k-config',
 'status': <Status.RUNNING: 'RUNNING'>,
 'details': {'resources': {'backend_details': {'model': {'name': 'gpt-4-32k',
     'version': 'latest'}}},
  'scaling': {'backend_details': {}}}}

## Use SAP Gen AI Hub SDK and instantiate one of the deployed Gen AI Models

In [8]:
from gen_ai_hub.proxy.langchain.init_models import init_llm

In [9]:
from gen_ai_hub.proxy.gen_ai_hub_proxy.client import GenAIHubProxyClient

genai_client=GenAIHubProxyClient(ai_core_client=ai_core_client)

In [10]:
# `init_llm` to explicitely reuse the `ai_core_client` you created above

llm = init_llm('gpt-4-32k', temperature=0., max_tokens=256, proxy_client=genai_client)

In [11]:
print(llm.proxy_client.ai_core_client.rest_client.base_url)
print(f"{llm.proxy_client.ai_core_client.rest_client.get_token()[:20]} ...")

https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
Bearer eyJhbGciOiJSU ...


In [12]:
# Execute some prompt on the LLM

llm_response=llm.invoke('Where is Wrocław located and what are top 5 places I should visit there?')

In [13]:
print(llm_response.content)
display(llm_response.response_metadata)

Wrocław is located in western Poland, on the banks of the River Oder. It is the largest city in the region of Lower Silesia.

Top 5 places to visit in Wrocław:

1. Market Square (Rynek): This is one of the largest market squares in Europe, filled with colorful buildings, restaurants, and shops. The Gothic-style Old Town Hall is a must-see.

2. Wrocław Cathedral (Cathedral of St. John the Baptist): This is a stunning example of Gothic architecture and the seat of the Roman Catholic Archdiocese of Wrocław. You can climb up to the viewing platform for a panoramic view of the city.

3. Wrocław's Dwarfs: Scattered throughout the city, these small figurines have become a symbol of Wrocław. There are over 350 dwarfs and finding them can be a fun activity for both kids and adults.

4. Panorama of Raclawice: This is a gigantic painting depicting the Battle of Raclawice during the Kosciuszko Uprising against Russia. The painting is displayed in a circular fashion creating a 360° view.

5. Centen

{'token_usage': {'completion_tokens': 256,
  'prompt_tokens': 25,
  'total_tokens': 281},
 'model_name': 'gpt-4-32k',
 'system_fingerprint': None,
 'finish_reason': 'length',
 'logprobs': None}

## That's it for now!